In [2]:
import pandas as pd
import json
import os

In [3]:
with open('export(5).geojson', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [4]:
data

{'type': 'FeatureCollection',
 'generator': 'overpass-turbo',
 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.',
 'timestamp': '2025-09-08T03:51:25Z',
 'features': [{'type': 'Feature',
   'properties': {'@id': 'relation/3425413',
    'amenity': 'library',
    'building': 'yes',
    'name': 'Perpustakaan UNRAM',
    'short_name': 'Perpustakaan UNRAM',
    'source': 'survey',
    'survey:date': '2013-12-10',
    'type': 'multipolygon'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[116.093099, -8.5867397],
      [116.092943, -8.5873204],
      [116.0933161, -8.5874184],
      [116.0934721, -8.5868377],
      [116.093099, -8.5867397]],
     [[116.0932185, -8.5869393],
      [116.0932652, -8.5869516],
      [116.0932521, -8.5870004],
      [116.0932054, -8.5869881],
      [116.0932185, -8.5869393]],
     [[116.0931697, -8.5871433],
      [116.0932185, -8.5871541],
      [116.0932083, -8.587199],
      [116.09

In [11]:
processed_features = []

    # Loop through each feature in the GeoJSON
for feature in data['features']:
        properties = feature.get('properties', {})
        geometry = feature.get('geometry', {})
        
        # Start with the properties dictionary
        row = properties.copy()
        
        # Add the main ID
        row['id'] = feature.get('id')
        
        # Extract geometry type and a representative coordinate
        if geometry:
            geom_type = geometry.get('type')
            coordinates = geometry.get('coordinates')
            
            row['geometry_type'] = geom_type
            
            # For simplicity, we extract the first coordinate pair
            # For a Point, it's the point itself.
            # For a Polygon, it's the first vertex of the outer ring.
            try:
                if geom_type == 'Point':
                    row['longitude'] = coordinates[0]
                    row['latitude'] = coordinates[1]
                elif geom_type == 'Polygon':
                    # Taking the first coordinate of the first ring
                    row['longitude'] = coordinates[0][0][0]
                    row['latitude'] = coordinates[0][0][1]
            except (IndexError, TypeError):
                row['longitude'] = None
                row['latitude'] = None
        
        processed_features.append(row)


In [12]:
# Create a pandas DataFrame
print("Creating DataFrame...")
df = pd.DataFrame(processed_features)

    # --- Data Cleaning ---
    # Remove the redundant '@id' column if it exists
if '@id' in df.columns:
    df = df.drop(columns=['@id'])
        
    # Reorder columns to make the most important ones appear first
    preferred_order = [
        'id', 'name', 'amenity', 'shop', 'latitude', 'longitude', 'geometry_type'
    ]
    # Get the remaining columns
    remaining_columns = [col for col in df.columns if col not in preferred_order]
    
    # Combine and set the new order
    new_column_order = preferred_order + sorted(remaining_columns)
    df = df[new_column_order]

Creating DataFrame...


In [13]:
df

,id,name,amenity,shop,latitude,longitude,geometry_type,access,access:roof,addr:city,...,website,website:menu,wheelchair,width,wikidata,wikimedia_commons,wikipedia,wikipedia:en,wikipedia:id,year_of_construction
0,relation/7129910,Yayasan Rumah Pendidikan Islam,school,NaN,-6.228136,106.834694,Polygon,NaN,no,DKI Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,relation/7190278,SMA Tarakanita 1 & SMK Tarakanita,school,NaN,-6.245390,106.812265,Polygon,NaN,no,DKI Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,relation/7272407,SD dan SMP Swasta YWKA I,school,NaN,-6.217491,106.851723,Polygon,NaN,no,DKI Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,relation/7272408,SD Muhammadiyah,school,NaN,-6.234509,106.853376,Polygon,NaN,yes,DKI Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,relation/7272857,SD Negeri Manggarai 1 Pagi/02 petang,school,NaN,-6.212616,106.855962,Polygon,NaN,no,DKI Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6100,node/13038504181,Rumah Makan Sinar Surya,restaurant,NaN,-6.241212,106.789321,Point,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6101,node/13038504187,BRI Mayestik,atm,NaN,-6.242026,106.790286,Point,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6102,node/13038504188,NaN,atm,NaN,-6.242018,106.790225,Point,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6103,node/13038504189,Mixue Kyai Maja,cafe,NaN,-6.242044,106.790441,Point,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df['amenity'].unique()

array(['school', 'college', 'place_of_worship', 'hospital', 'fuel',
       'parking', 'bbq', 'grave_yard', 'bank', 'post_office', 'clinic',
       'community_centre', 'studio', 'university', nan, 'kindergarten',
       'cafe', 'restaurant', 'car_pooling', 'marketplace', 'car_wash',
       'police', 'library', 'courthouse', 'fire_station',
       'motorcycle_parking', 'doctors', 'fast_food', 'pharmacy',
       'dentist', 'food_court', 'toilets', 'mall', 'parking_space',
       'fountain', 'cinema', 'bar', 'townhall', 'pub', 'internet_cafe',
       'social_facility', 'nightclub', 'animal_boarding', 'bus_station',
       'shelter', 'arts_centre', 'prep_school', 'biergarten', 'prison',
       'events_venue', 'language_school', 'bicycle_rental',
       'waste_transfer_station', 'compressed_air', 'payment_centre',
       'conference_centre', 'security_booth', 'taxi', 'atm',
       'fast_food;cafe', 'telephone', 'parking_entrance', 'ice_cream',
       'bench', 'grit_bin', 'car_rental', 'burea

In [ ]:
# marketplace fast_food restaurant theatre studio cafe pub

# food_court toko bangunan

In [10]:
with open('jakartaSelatan.geojson', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [2]:
import pandas as pd

In [3]:
clean = pd.read_csv('test.csv')

/tmp/ipykernel_7372/3198630040.py:1: DtypeWarning: Columns (7,9,11,13,14,16,17,18,19,20,21,26,29,30,31,32,33,34,35,36,38,39,40,41,44,45,46,47,48,49,51,52,53,55,58,60,63,64,66,67,68,69,70,71,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,118,119,120,121,122,123,124,125,126,128,129,130,134,135,136,137,138,139,140,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,159,160,161,162,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,193,194,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,218,219,220,221,223,224,225,226,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,275,276,277,278,279,281,282,283,284,285,287,288,289,290,291,292,293,294,297,298,299,301,303,3

In [8]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4139 entries, 0 to 4138
Columns: 362 entries, id to wikipedia:en
dtypes: float64(15), object(347)
memory usage: 11.4+ MB


In [10]:
clean_df = pd.read_csv('kabupaten_jakarta_pois_categorized.csv')

/tmp/ipykernel_7372/3224022249.py:1: DtypeWarning: Columns (6,8,10,12,13,15,16,17,18,19,20,25,28,29,30,31,32,33,34,35,37,38,39,40,43,44,45,46,47,48,50,51,52,54,57,59,62,63,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,117,118,119,120,121,122,123,124,125,127,128,129,133,134,135,136,137,138,139,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,157,158,159,160,161,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,193,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,274,275,276,277,278,280,281,282,283,284,286,287,288,289,290,291,292,293,296,297,298,300,302,30

In [ ]:
clean_df = clean_df[['id','kabupaten','name','category','latitude','longitude','addr:city','source']]

In [15]:
clean_df['category'].unique()

array(['supermarket', 'atm', 'restaurant', 'convenience', 'hairdresser',
       'cinema', 'bakery', 'mall', 'travel_agency', 'fast_food', 'bar',
       'farm', 'pet', 'greengrocer', 'beauty', 'funeral_directors',
       'books', 'mobile_phone', 'beverages', 'ticket', 'gift',
       'parking_entrance', 'clinic', 'university', 'place_of_worship',
       'laundry', 'shoes', 'massage', 'motorcycle', 'stationery',
       'car_repair', 'variety_store', 'optician', 'cafe',
       'department_store', 'houseware', 'tailor', 'chocolate', 'clothes',
       'car_parts', 'car', 'music_school', 'wine', 'sports', 'vacant',
       'food_court', 'computer', 'copyshop', 'pastry', 'bank', 'tyres',
       'photo', 'furniture', 'pawnbroker', 'electronics', 'hardware',
       'confectionery', 'florist', 'tea', 'kiosk', 'doityourself',
       'radiotechnics', 'rental', 'pastry;seafood', 'pharmacy',
       'telecommunication', 'nutrition_supplements', 'rice', 'fuel',
       'bicycle_parking', 'coffee', 'music

In [17]:
clean_df.to_csv('data_poi/jakarta_poi.csv')

### test read

In [18]:
kabupaten = pd.read_csv('data_Internet/telecom_data_with_kode_kabupaten_irul.csv')

In [19]:
kabupaten


,Kode_Kabupaten,PROVINSI,KABUPATEN,5G/4G/LTE_2021,3G/H/H+/EVDO_2021,"2,5G/E/GPRS_2021",5G/4G/LTE_2024,3G/H/H+/EVDO_2024,"2,5G/E/GPRS_2024"
0,11.09,ACEH,Simeulue,102.0,17.0,11.0,128,3.0,3.0
1,11.10,ACEH,Aceh Singkil,89.0,24.0,5.0,107,11.0,1.0
2,11.01,ACEH,Aceh Selatan,187.0,60.0,5.0,240,16.0,1.0
3,11.02,ACEH,Aceh Tenggara,313.0,49.0,5.0,358,13.0,3.0
4,11.03,ACEH,Aceh Timur,349.0,125.0,23.0,426,62.0,11.0
...,...,...,...,...,...,...,...,...,...
509,82.06,MALUKU UTARA,Halmahera Timur,NaN,NaN,NaN,79,6.0,17.0
510,82.07,MALUKU UTARA,Pulau Morotai,NaN,NaN,NaN,75,4.0,6.0
511,82.08,MALUKU UTARA,Pulau Taliabu,NaN,NaN,NaN,64,6.0,0.0
512,82.71,MALUKU UTARA,Kota Ternate,NaN,NaN,NaN,66,5.0,4.0


In [20]:
listK = kabupaten['KABUPATEN']

In [22]:
listK.info()

<class 'pandas.core.series.Series'>
RangeIndex: 514 entries, 0 to 513
Series name: KABUPATEN
Non-Null Count  Dtype 
--------------  ----- 
514 non-null    object
dtypes: object(1)
memory usage: 4.1+ KB


In [23]:
test_20_df = kabupaten.head(20)

In [24]:
test_20_df.to_csv('20dataTest.csv')

### jawa timur type shii


In [26]:
jatim_df = kabupaten.loc[kabupaten['PROVINSI'] == 'JAWA TIMUR'] 

In [28]:
jatim_df.to_csv('jatim_df.csv')

In [30]:
pulau_jawa = kabupaten.loc[
    (kabupaten['PROVINSI'] == 'JAWA TIMUR') | 
    (kabupaten['PROVINSI'] == 'JAWA BARAT') | 
    (kabupaten['PROVINSI'] == 'JAWA TENGAH') | 
    (kabupaten['PROVINSI'] == 'YOGYAKARTA') | 
    (kabupaten['PROVINSI'] == 'BANTEN')
]

In [32]:
pulau_jawa.to_csv('pulau_jawa.csv')

In [33]:
pulau_jawa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113 entries, 32 to 177
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Kode_Kabupaten     113 non-null    float64
 1   PROVINSI           113 non-null    object 
 2   KABUPATEN          113 non-null    object 
 3   5G/4G/LTE_2021     113 non-null    float64
 4   3G/H/H+/EVDO_2021  113 non-null    float64
 5   2,5G/E/GPRS_2021   113 non-null    float64
 6   5G/4G/LTE_2024     113 non-null    int64  
 7   3G/H/H+/EVDO_2024  113 non-null    float64
 8   2,5G/E/GPRS_2024   113 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 8.8+ KB


In [37]:
jawa1 = pulau_jawa.head(1)

In [38]:
jawa1.to_csv('1.csv')

In [39]:
pulau_sumatera = kabupaten.loc[
    (kabupaten['PROVINSI'] == 'SUMATERA UTARA') | 
    (kabupaten['PROVINSI'] == 'SUMATERA BARAT') | 
    (kabupaten['PROVINSI'] == 'RIAU') | 
    (kabupaten['PROVINSI'] == 'KEP. RIAU') | 
    (kabupaten['PROVINSI'] == 'JAMBI') | 
    (kabupaten['PROVINSI'] == 'SUMATERA SELATAN') | 
    (kabupaten['PROVINSI'] == 'BANGKA BELITUNG') | 
    (kabupaten['PROVINSI'] == 'LAMPUNG') |
    (kabupaten['PROVINSI'] == 'ACEH')
]

In [40]:
pulau_sumatera.to_csv('pulau_sumatera.csv')

In [41]:
pulau_kalimantan = kabupaten.loc[
    (kabupaten['PROVINSI'] == 'KALIMANTAN BARAT') | 
    (kabupaten['PROVINSI'] == 'KALIMANTAN TIMUR') | 
    (kabupaten['PROVINSI'] == 'KALIMANTAN SELATAN') | 
    (kabupaten['PROVINSI'] == 'KALIMANTAN UTARA') | 
   (kabupaten['PROVINSI'] == 'KALIMANTAN TENGAH')
]

In [42]:
pulau_kalimantan.to_csv('pulau_kalimantan.csv')

In [43]:
pulau_sulawesi = kabupaten.loc[
    (kabupaten['PROVINSI'] == 'SULAWESI BARAT') | 
    (kabupaten['PROVINSI'] == 'SULAWESI TENGGARA') | 
    (kabupaten['PROVINSI'] == 'SULAWESI SELATAN') | 
    (kabupaten['PROVINSI'] == 'SULAWESI UTARA') | 
   (kabupaten['PROVINSI'] == 'SULAWESI TENGAH')| 
   (kabupaten['PROVINSI'] == 'GORONTALO')
]

In [44]:
pulau_sulawesi.to_csv('pulau_sulawesi.csv')

In [45]:
pulau_papua = kabupaten.loc[
    (kabupaten['PROVINSI'] == 'PAPUA BARAT') | 
    (kabupaten['PROVINSI'] == 'PAPUA') | 
    (kabupaten['PROVINSI'] == 'PAPUA BARAT DAYA') | 
    (kabupaten['PROVINSI'] == 'PAPUA SELATAN') | 
   (kabupaten['PROVINSI'] == 'PAPUA TENGAH')| 
   (kabupaten['PROVINSI'] == 'PAPUA PEGUNUNGAN')
]

In [46]:
pulau_papua.to_csv('pulau_papua.csv')

In [47]:
ntb_other_pulau = kabupaten.loc[
    (kabupaten['PROVINSI'] == 'KEP. BANGKA BELITUNG') | 
    (kabupaten['PROVINSI'] == 'BALI') | 
    (kabupaten['PROVINSI'] == 'MALUKU') | 
    (kabupaten['PROVINSI'] == 'MALUKU UTARA') | 
   (kabupaten['PROVINSI'] == 'NUSA TENGGARA BARAT')| 
   (kabupaten['PROVINSI'] == 'NUSA TENGGARA TIMUR')
]

In [49]:
ntb_other_pulau.to_csv('ntb_other_pulau.csv')

# Cleaning all df


## Upload DF

In [4]:
jakarta = pd.read_csv('kabupaten_jakarta_pois_categorized.csv')
sumatra = pd.read_csv('pulau_sumatera_categorized.csv')
kalimantan = pd.read_csv('pulau_kalimantan_categorized.csv')
sulawesi = pd.read_csv('pulau_sulawesi_categorized.csv')
jawa = pd.read_csv('jawa.csv')
papua = pd.read_csv('pulau_papua_categorized.csv')
ntb = pd.read_csv('ntb_other_pulau_categorized.csv')
other = pd.read_csv('failed_list_retry_categorized.csv')

/tmp/ipykernel_107303/3297057775.py:1: DtypeWarning: Columns (6,8,10,12,13,15,16,17,18,19,20,25,28,29,30,31,32,33,34,35,37,38,39,40,43,44,45,46,47,48,50,51,52,54,57,59,62,63,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,117,118,119,120,121,122,123,124,125,127,128,129,133,134,135,136,137,138,139,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,157,158,159,160,161,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,193,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,274,275,276,277,278,280,281,282,283,284,286,287,288,289,290,291,292,293,296,297,298,300,302,

In [7]:
jakarta = jakarta[['id','kabupaten','name','category','latitude','longitude','addr:city','source']]
sumatra = sumatra[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]
kalimantan = kalimantan[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]
sulawesi = sulawesi[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]
jawa = jawa[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]
papua = papua[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]
ntb = ntb[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]
other = other[['id','kabupaten_kota','name','category','latitude','longitude','addr:city','source']]

In [13]:
jakarta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4139 entries, 0 to 4138
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4139 non-null   object 
 1   kabupaten_kota  4139 non-null   object 
 2   name            3213 non-null   object 
 3   category        4139 non-null   object 
 4   latitude        3169 non-null   float64
 5   longitude       3169 non-null   float64
 6   addr:city       716 non-null    object 
 7   source          176 non-null    object 
dtypes: float64(2), object(6)
memory usage: 258.8+ KB


In [14]:
jakarta.rename(columns={'kabupaten':'kabupaten_kota'}, inplace=True)


In [18]:
ntb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11034 entries, 0 to 11033
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11034 non-null  object 
 1   kabupaten_kota  11034 non-null  object 
 2   name            7911 non-null   object 
 3   category        11034 non-null  object 
 4   latitude        7468 non-null   float64
 5   longitude       7468 non-null   float64
 6   addr:city       1669 non-null   object 
 7   source          874 non-null    object 
dtypes: float64(2), object(6)
memory usage: 689.8+ KB


## Merge


In [19]:
result = pd.concat([jakarta, sumatra, sulawesi, kalimantan, jawa, ntb, papua, other], axis=0)

# Reset index if needed
result = result.reset_index(drop=True)

In [21]:
result.to_csv('poi_osm/fix/osm_poi_indo.csv')

In [23]:
len(result['kabupaten_kota'].unique())

381

In [1]:
import pandas as pd
import os

# 1. Tentukan path ke folder utama yang berisi file-file CSV
folder_path = "poi_gmaps"  # Ganti dengan path lengkap jika perlu, e.g., "C:/project/data"

# 2. Dapatkan daftar semua file CSV dalam folder
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# (Opsional) Urutkan nama file jika urutan penggabungan penting
all_files.sort()

# 3. Inisialisasi list untuk menyimpan setiap DataFrame
list_of_dfs = []

# 4. Loop melalui setiap file CSV, baca, dan tambahkan ke list
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    try:
        # Baca file CSV menjadi DataFrame
        df = pd.read_csv(file_path)
        # (Opsional) Tambahkan kolom baru untuk melacak sumber data
        df['source_file'] = filename
        # Tambahkan DataFrame ke list
        list_of_dfs.append(df)
        print(f"Berhasil membaca: {filename} (Shape: {df.shape})")
    except Exception as e:
        print(f"Error membaca file {filename}: {e}")

# 5. Gabungkan semua DataFrame yang telah dikumpulkan
if list_of_dfs:
    # Gabungkan secara vertikal (ignore_index membuat index baru yang bersih)
    final_df = pd.concat(list_of_dfs, ignore_index=True)
    
    # 6. Tampilkan informasi tentang DataFrame gabungan
    print("\n" + "="*50)
    print("PENGgabungan SELESAI!")
    print(f"Total file yang digabung: {len(list_of_dfs)}")
    print(f"Shape DataFrame Gabungan: {final_df.shape}")
    print("\n5 Baris pertama:")
    print(final_df.head())
    
    # 7. (Opsional) Simpan hasil gabungan ke file baru
    output_path = "data/gabungan_semua_data.csv"
    final_df.to_csv(output_path, index=False)
    print(f"\nData gabungan disimpan sebagai: {output_path}")
    
else:
    print("Tidak ada file CSV yang ditemukan di folder.")

Berhasil membaca: gmaps_poi_results_Kabupaten Aceh Jaya.csv (Shape: (453, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Anambas.csv (Shape: (132, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Asmat.csv (Shape: (153, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Balangan.csv (Shape: (445, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Bangka Barat.csv (Shape: (564, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Banyuwangi.csv (Shape: (334, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Blora.csv (Shape: (626, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Boven Digoel.csv (Shape: (260, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Buru Selatan.csv (Shape: (404, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Buton Selatan.csv (Shape: (569, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Gunung Mas.csv (Shape: (457, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Gunungkidul.csv (Shape: (696, 7))
Berhasil membaca: gmaps_poi_results_Kabupaten Halmahera Ba

In [2]:
data = pd.read_csv('fix_gmaps_poi_results_Jakarta Pusat.csv')

In [3]:
data

,category,name,address,coordinates
0,Mall,Mal Atrium Senen,"Jl. Senen Raya No.135, Senen, Kec. Senen, Kota...","-6.177522, 106.8415448"
1,Mall,Grand Indonesia,"Jl. M.H. Thamrin No.1, Menteng, Kec. Menteng, ...","-6.1951984, 106.8217199"
2,Mall,Mall Senayan City,"Jl. Asia Afrika No.Lot 19, Gelora, Kecamatan T...","-6.2271953, 106.7970117"
3,Mall,Plaza Indonesia,"Jl. M.H. Thamrin No.Kav.28-30, Gondangdia, Kec...","-6.1935977, 106.8219468"
4,Mall,Mall Plaza Senayan,"Jl. Asia Afrika No.8, RT.1/RW.3, Gelora, Kecam...","-6.2254775, 106.7992649"
...,...,...,...,...
479,pasar,Pasar Mencos,"Jl. Karet Belakang Barat No.2 11, RT.11/RW.4, ...","-6.2137623, 106.8259789"
480,pasar,Pasar Petak Sembilan,"Jl. Kemenangan Raya No.40 5, RT.5/RW.1, Glodok...","-6.1438308, 106.8128989"
481,pasar,Pusat Grosir Central Tanah Abang (CTA),"Jl. Kb. Jati No.5a, RT.1/RW.9, Kb. Kacang, Kec...","-6.1889002, 106.8124168"
482,pasar,Pasar Pagi Asemka,"1, RT.11/RW.1, Roa Malaka, Kec. Tambora, Kota ...","-6.1404628, 106.8120538"


In [6]:
data['kabupaten'] = 'Jakarta Pusat'
data['provinsi'] = 'DKI Jakarta'

In [12]:
data = data[['kabupaten', 'provinsi', 'category', 'name', 'address','coordinates']]

In [ ]:
final = pd.read_csv('/home/rul/Kuliah/Scrapping_pelita/poi_gmaps/fix/gabungan_semua_data.csv')

In [9]:
final.head(2)

,kabupaten,provinsi,category,name,address,coordinates,source_file
0,Kabupaten Aceh Jaya,Aceh,Mall,Sultan Swalayan,"JHMJ+9VM, Dayah Baro, Kec. Krueng Sabee, Kabup...","4.6334625, 95.5821719",gmaps_poi_results_Kabupaten Aceh Jaya.csv
1,Kabupaten Aceh Jaya,Aceh,Mall,Suzuya Mall Banda Aceh,"Jl. Teuku Umar, Lamtemen Tim., Kec. Jaya Baru,...","5.5377324, 95.3068005",gmaps_poi_results_Kabupaten Aceh Jaya.csv


In [14]:
data.head(2)

,kabupaten,provinsi,category,name,address,coordinates
0,Jakarta Pusat,DKI Jakarta,Mall,Mal Atrium Senen,"Jl. Senen Raya No.135, Senen, Kec. Senen, Kota...","-6.177522, 106.8415448"
1,Jakarta Pusat,DKI Jakarta,Mall,Grand Indonesia,"Jl. M.H. Thamrin No.1, Menteng, Kec. Menteng, ...","-6.1951984, 106.8217199"


In [16]:
df_merged = pd.concat([final, data], ignore_index=True, sort=False)

In [18]:
df_merged.to_csv('poi_gmaps/fix/gmaps_withsource.csv')

In [20]:
df_merged.head(2)

,kabupaten,provinsi,category,name,address,coordinates,source_file
0,Kabupaten Aceh Jaya,Aceh,Mall,Sultan Swalayan,"JHMJ+9VM, Dayah Baro, Kec. Krueng Sabee, Kabup...","4.6334625, 95.5821719",gmaps_poi_results_Kabupaten Aceh Jaya.csv
1,Kabupaten Aceh Jaya,Aceh,Mall,Suzuya Mall Banda Aceh,"Jl. Teuku Umar, Lamtemen Tim., Kec. Jaya Baru,...","5.5377324, 95.3068005",gmaps_poi_results_Kabupaten Aceh Jaya.csv


In [21]:
df_merged = df_merged[['kabupaten', 'provinsi', 'category', 'name', 'address','coordinates']]

In [25]:
df_merged.to_csv('poi_gmaps/fix/fix_gmaps_poi.csv')

In [26]:
cleaning = pd.read_csv('/home/rul/Kuliah/Scrapping_pelita/poi_gmaps/fix/fix_gmaps_poi.csv')

In [28]:
cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37043 entries, 0 to 37042
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   37043 non-null  int64 
 1   kabupaten    37043 non-null  object
 2   provinsi     37043 non-null  object
 3   category     37043 non-null  object
 4   name         37042 non-null  object
 5   address      37043 non-null  object
 6   coordinates  37017 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.0+ MB


In [30]:
# Cek contoh data duplikat untuk investigasi
duplicate_coords = cleaning[cleaning.duplicated(subset=['coordinates'], keep=False)]

# Tampilkan beberapa contoh duplikat
print("Contoh data duplikat berdasarkan coordinates:")
print(duplicate_coords.sort_values('coordinates').head(10)[['name', 'kabupaten', 'provinsi', 'coordinates']])

Contoh data duplikat berdasarkan coordinates:
                  name       kabupaten          provinsi  \
32019     Fresh Mart 4  Kota Pontianak  Kalimantan Barat   
31617     Fresh Mart 4  Kota Pontianak  Kalimantan Barat   
31504  Toko Mas Safari  Kota Pontianak  Kalimantan Barat   
31496  Toko Mas Safari  Kota Pontianak  Kalimantan Barat   
31984    Pasar Teratai  Kota Pontianak  Kalimantan Barat   
31420    Pasar Teratai  Kota Pontianak  Kalimantan Barat   
31589       Mitra Anda  Kota Pontianak  Kalimantan Barat   
31872       Mitra Anda  Kota Pontianak  Kalimantan Barat   
31600       Mini Super  Kota Pontianak  Kalimantan Barat   
31938       Mini Super  Kota Pontianak  Kalimantan Barat   

                   coordinates  
32019  -0.0054459, 109.3068758  
31617  -0.0054459, 109.3068758  
31504   -0.006381, 109.3118003  
31496   -0.006381, 109.3118003  
31984  -0.0068447, 109.3115897  
31420  -0.0068447, 109.3115897  
31589  -0.0187974, 109.3270569  
31872  -0.0187974, 109.327056

In [31]:
# Cek apakah ada coordinates yang sama tapi punya kabupaten berbeda
coord_kabupaten_conflict = duplicate_coords.groupby('coordinates')['kabupaten'].nunique()
conflict_coords = coord_kabupaten_conflict[coord_kabupaten_conflict > 1]

print(f"Jumlah coordinates dengan kabupaten berbeda: {len(conflict_coords)}")

Jumlah coordinates dengan kabupaten berbeda: 334


In [32]:
# Langkah komprehensif untuk cleaning
def clean_duplicate_data(df):
    # 1. Drop rows dengan coordinates missing
    df_clean = df.dropna(subset=['coordinates']).copy()
    
    # 2. Identifikasi coordinates unik
    unique_coords = df_clean['coordinates'].unique()
    
    # 3. Untuk setiap unique coordinate, ambil record pertama
    cleaned_data = df_clean.drop_duplicates(subset=['coordinates'], keep='first')
    
    return cleaned_data

# Eksekusi cleaning
df_cleaned = clean_duplicate_data(cleaning)

In [33]:
print("=== HASIL CLEANING ===")
print(f"Total data awal: {len(cleaning)}")
print(f"Data setelah cleaning: {len(df_cleaned)}")
print(f"Data yang dihapus: {len(cleaning) - len(df_cleaned)}")
print(f"Persentase data retained: {len(df_cleaned)/len(cleaning)*100:.2f}%")

# Cek distribusi setelah cleaning
print("\nDistribusi provinsi setelah cleaning:")
print(df_cleaned['provinsi'].value_counts().head())

=== HASIL CLEANING ===
Total data awal: 37043
Data setelah cleaning: 32208
Data yang dihapus: 4835
Persentase data retained: 86.95%

Distribusi provinsi setelah cleaning:
provinsi
Sulawesi Barat      1205
Bali                1164
Banten              1141
Sumatera Selatan    1140
Jawa Barat          1129
Name: count, dtype: int64


In [34]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32208 entries, 0 to 37042
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   32208 non-null  int64 
 1   kabupaten    32208 non-null  object
 2   provinsi     32208 non-null  object
 3   category     32208 non-null  object
 4   name         32208 non-null  object
 5   address      32208 non-null  object
 6   coordinates  32208 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.0+ MB


In [35]:
df_cleaned = df_cleaned[['kabupaten', 'provinsi', 'category', 'name', 'address','coordinates']]

In [37]:
df_cleaned.to_csv('poi_gmaps/fix/fix_gmaps_poi.csv')

### Merge 2 dataset for POI


In [79]:
osm = pd.read_csv('/home/rul/Kuliah/Scrapping_pelita/poi_osm/fix/osm_poi_indo.csv')

In [80]:
gmaps = pd.read_csv('/home/rul/Kuliah/Scrapping_pelita/poi_gmaps/fix/fix_gmaps_poi.csv')

In [81]:
kode = pd.read_csv('/home/rul/Kuliah/Scrapping_pelita/data_Internet/sinyal_kabupaten_irul.csv')

In [82]:
kode.head

<bound method NDFrame.head of      Kode_Kabupaten      PROVINSI              KABUPATEN  5G/4G/LTE_2021  \
0             11.09          ACEH               Simeulue           102.0   
1             11.10          ACEH           Aceh Singkil            89.0   
2             11.01          ACEH           Aceh Selatan           187.0   
3             11.02          ACEH          Aceh Tenggara           313.0   
4             11.03          ACEH             Aceh Timur           349.0   
..              ...           ...                    ...             ...   
509           82.06  MALUKU UTARA        Halmahera Timur             NaN   
510           82.07  MALUKU UTARA          Pulau Morotai             NaN   
511           82.08  MALUKU UTARA          Pulau Taliabu             NaN   
512           82.71  MALUKU UTARA           Kota Ternate             NaN   
513           82.72  MALUKU UTARA  Kota Tidore Kepulauan             NaN   

     3G/H/H+/EVDO_2021  2,5G/E/GPRS_2021  5G/4G/LTE_2024 

In [83]:
gmaps

,Unnamed: 0,kabupaten,provinsi,category,name,address,coordinates
0,0,Kabupaten Aceh Jaya,Aceh,Mall,Sultan Swalayan,"JHMJ+9VM, Dayah Baro, Kec. Krueng Sabee, Kabup...","4.6334625, 95.5821719"
1,1,Kabupaten Aceh Jaya,Aceh,Mall,Suzuya Mall Banda Aceh,"Jl. Teuku Umar, Lamtemen Tim., Kec. Jaya Baru,...","5.5377324, 95.3068005"
2,2,Kabupaten Aceh Jaya,Aceh,Mall,Komplek Pasar Calang,"JHPM+X75, Unnamed Rd, Sentosa, Kec. Krueng Sab...","4.6373826, 95.5832344"
3,3,Kabupaten Aceh Jaya,Aceh,Mall,Pangkalan Barata Jaya,"48PQ+5MP, Unnamed Road, Meudheun, Jaya, Kabupa...","5.1354551, 95.3392001"
4,4,Kabupaten Aceh Jaya,Aceh,Mall,sultan swalayan,"Jl. Meulaboh - Banda Aceh, Panton, Kec. Teunom...","4.445264, 95.8123024"
...,...,...,...,...,...,...,...
32203,37036,Jakarta Pusat,DKI Jakarta,pasar,Pasar,"Jl. Tanah Tinggi XII No.15 23, RT.5/RW.9, Tana...","-6.1815939, 106.8512996"
32204,37038,Jakarta Pusat,DKI Jakarta,pasar,Pasar Mencos,"Jl. Karet Belakang Barat No.2 11, RT.11/RW.4, ...","-6.2137623, 106.8259789"
32205,37039,Jakarta Pusat,DKI Jakarta,pasar,Pasar Petak Sembilan,"Jl. Kemenangan Raya No.40 5, RT.5/RW.1, Glodok...","-6.1438308, 106.8128989"
32206,37041,Jakarta Pusat,DKI Jakarta,pasar,Pasar Pagi Asemka,"1, RT.11/RW.1, Roa Malaka, Kec. Tambora, Kota ...","-6.1404628, 106.8120538"


In [84]:
gmaps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32208 entries, 0 to 32207
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   32208 non-null  int64 
 1   kabupaten    32208 non-null  object
 2   provinsi     32208 non-null  object
 3   category     32208 non-null  object
 4   name         32208 non-null  object
 5   address      32208 non-null  object
 6   coordinates  32208 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.7+ MB


In [85]:
import pandas as pd
import numpy as np

# 1. Daftar mapping untuk mengubah nama kabupaten di gmaps
nama_mapping = {
    'Kota Sofifi': 'Kota Tidore Kepulauan',
    'Tanjung Selor': 'Bulungan',
    'Kabupaten Anambas': 'Kepulauan Anambas'
}

# 2. Apply perubahan nama kabupaten di df gmaps
print("Sebelum perubahan:")
print(gmaps['kabupaten'].value_counts().loc[list(nama_mapping.keys())])

gmaps['kabupaten'] = gmaps['kabupaten'].replace(nama_mapping)

print("\nSetelah perubahan:")
print(gmaps['kabupaten'].value_counts().loc[list(nama_mapping.values())])

# 3. Sekarang lakukan matching Kode_Kabupaten dengan nama yang sudah diperbaiki
def standardize_kabupaten_name(name):
    """
    Standardize kabupaten name untuk matching yang lebih baik
    """
    if pd.isna(name):
        return name
    
    name = name.upper().strip()
    
    # Handle common variations
    name = name.replace('KABUPATEN', '').replace('KAB.', '').strip()
    name = name.replace('KOTA ', '').strip()
    name = name.replace('ADM. ', '').strip()
    
    # Remove extra spaces
    name = ' '.join(name.split())
    
    return name

# 4. Apply standardization
kode['kabupaten_clean'] = kode['KABUPATEN'].apply(standardize_kabupaten_name)
gmaps['kabupaten_clean'] = gmaps['kabupaten'].apply(standardize_kabupaten_name)

# 5. Pastikan Kode_Kabupaten sebagai string untuk konsistensi
kode['Kode_Kabupaten'] = kode['Kode_Kabupaten'].astype(str)

# 6. Buat mapping dictionary dari dataframe kode
kode_mapping = kode.set_index('kabupaten_clean')['Kode_Kabupaten'].to_dict()

# 7. Tambahkan Kode_Kabupaten ke gmaps menggunakan mapping
gmaps['Kode_Kabupaten'] = gmaps['kabupaten_clean'].map(kode_mapping)

# 8. Handle kasus khusus lainnya
manual_mapping = {
    'JAKARTA PUSAT': '31.71',
    'JAKARTA SELATAN': '31.74',
    'JAKARTA BARAT': '31.73',
    'JAKARTA TIMUR': '31.75',
    'JAKARTA UTARA': '31.72',
    'KEPULAUAN SERIBU': '31.01',
}

# 9. Apply manual mapping untuk data yang masih tidak match
for kabupaten_clean, kode_kab in manual_mapping.items():
    mask = (gmaps['kabupaten_clean'] == kabupaten_clean) & (gmaps['Kode_Kabupaten'].isna())
    gmaps.loc[mask, 'Kode_Kabupaten'] = kode_kab

# 10. Hapus kolom temporary
gmaps = gmaps.drop(columns=['kabupaten_clean'])

# 11. Konversi Kode_Kabupaten ke string untuk konsistensi
gmaps['Kode_Kabupaten'] = gmaps['Kode_Kabupaten'].astype(str)

# 12. Atur urutan kolom
column_order = ['Unnamed: 0', 'kabupaten', 'Kode_Kabupaten', 'provinsi', 'category', 'name', 'address', 'coordinates']
gmaps = gmaps[column_order]

# 13. Tampilkan hasil akhir
print("\n=== HASIL AKHIR ===")
print(gmaps.info())
print(f"\nSuccess rate: {gmaps['Kode_Kabupaten'].notna().sum()/len(gmaps)*100:.2f}%")
print(f"Data tanpa Kode_Kabupaten: {gmaps['Kode_Kabupaten'].isna().sum()}")

# 14. Tampilkan perubahan nama yang dilakukan
print("\n=== PERUBAHAN NAMA YANG DILAKUKAN ===")
for old_name, new_name in nama_mapping.items():
    count_old = (gmaps['kabupaten'] == old_name).sum()
    count_new = (gmaps['kabupaten'] == new_name).sum()
    print(f"{old_name} → {new_name}: {count_new} data")

# 15. Verifikasi data untuk kabupaten yang diubah
print("\n=== VERIFIKASI DATA ===")
verified_kabupaten = list(nama_mapping.values())
for kabupaten in verified_kabupaten:
    data = gmaps[gmaps['kabupaten'] == kabupaten]
    if not data.empty:
        kode_kab = data['Kode_Kabupaten'].iloc[0]
        count = len(data)
        print(f"✓ {kabupaten}: {count} data → Kode: {kode_kab}")
    else:
        print(f"✗ {kabupaten}: Tidak ditemukan data")

# 16. Tampilkan sample data
print("\n=== SAMPLE DATA SETELAH PERUBAHAN ===")
sample_data = gmaps[gmaps['kabupaten'].isin(nama_mapping.values())]
if not sample_data.empty:
    print(sample_data[['kabupaten', 'Kode_Kabupaten', 'provinsi']].head(10))
else:
    print("Tidak ada data sample untuk kabupaten yang diubah")

Sebelum perubahan:
kabupaten
Kota Sofifi          200
Tanjung Selor        416
Kabupaten Anambas    114
Name: count, dtype: int64

Setelah perubahan:
kabupaten
Kota Tidore Kepulauan    200
Bulungan                 416
Kepulauan Anambas        114
Name: count, dtype: int64

=== HASIL AKHIR ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32208 entries, 0 to 32207
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      32208 non-null  int64 
 1   kabupaten       32208 non-null  object
 2   Kode_Kabupaten  32208 non-null  object
 3   provinsi        32208 non-null  object
 4   category        32208 non-null  object
 5   name            32208 non-null  object
 6   address         32208 non-null  object
 7   coordinates     32208 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.0+ MB
None

Success rate: 100.00%
Data tanpa Kode_Kabupaten: 0

=== PERUBAHAN NAMA YANG DILAKUKAN ===
Kota Sofifi 

In [88]:
gmaps = gmaps[['Kode_Kabupaten','kabupaten', 'provinsi', 'category', 'name', 'address','coordinates']]

In [91]:
gmaps

,Kode_Kabupaten,kabupaten,provinsi,category,name,address,coordinates
0,11.14,Kabupaten Aceh Jaya,Aceh,Mall,Sultan Swalayan,"JHMJ+9VM, Dayah Baro, Kec. Krueng Sabee, Kabup...","4.6334625, 95.5821719"
1,11.14,Kabupaten Aceh Jaya,Aceh,Mall,Suzuya Mall Banda Aceh,"Jl. Teuku Umar, Lamtemen Tim., Kec. Jaya Baru,...","5.5377324, 95.3068005"
2,11.14,Kabupaten Aceh Jaya,Aceh,Mall,Komplek Pasar Calang,"JHPM+X75, Unnamed Rd, Sentosa, Kec. Krueng Sab...","4.6373826, 95.5832344"
3,11.14,Kabupaten Aceh Jaya,Aceh,Mall,Pangkalan Barata Jaya,"48PQ+5MP, Unnamed Road, Meudheun, Jaya, Kabupa...","5.1354551, 95.3392001"
4,11.14,Kabupaten Aceh Jaya,Aceh,Mall,sultan swalayan,"Jl. Meulaboh - Banda Aceh, Panton, Kec. Teunom...","4.445264, 95.8123024"
...,...,...,...,...,...,...,...
32203,31.71,Jakarta Pusat,DKI Jakarta,pasar,Pasar,"Jl. Tanah Tinggi XII No.15 23, RT.5/RW.9, Tana...","-6.1815939, 106.8512996"
32204,31.71,Jakarta Pusat,DKI Jakarta,pasar,Pasar Mencos,"Jl. Karet Belakang Barat No.2 11, RT.11/RW.4, ...","-6.2137623, 106.8259789"
32205,31.71,Jakarta Pusat,DKI Jakarta,pasar,Pasar Petak Sembilan,"Jl. Kemenangan Raya No.40 5, RT.5/RW.1, Glodok...","-6.1438308, 106.8128989"
32206,31.71,Jakarta Pusat,DKI Jakarta,pasar,Pasar Pagi Asemka,"1, RT.11/RW.1, Roa Malaka, Kec. Tambora, Kota ...","-6.1404628, 106.8120538"


In [92]:
gmaps.to_csv('poi_gmaps/fix/poi_gmaps_with_kode.csv')

### OSM Kode

In [94]:
osm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56166 entries, 0 to 56165
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      56166 non-null  int64  
 1   id              56166 non-null  object 
 2   kabupaten_kota  56166 non-null  object 
 3   name            44723 non-null  object 
 4   category        56166 non-null  object 
 5   latitude        29646 non-null  float64
 6   longitude       29646 non-null  float64
 7   addr:city       5614 non-null   object 
 8   source          1830 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.9+ MB


In [95]:
osm

,Unnamed: 0,id,kabupaten_kota,name,category,latitude,longitude,addr:city,source
0,0,node/411668893,Jakarta Barat,Hero,supermarket,-6.165670,106.759241,NaN,NaN
1,1,node/412686079,Jakarta Barat,Super Indo,supermarket,-6.165414,106.782067,NaN,NaN
2,2,node/1354554837,Jakarta Barat,NaN,atm,-6.189133,106.796257,NaN,NaN
3,3,node/1354554844,Jakarta Barat,CFC,restaurant,-6.189235,106.796378,NaN,NaN
4,4,node/1541706845,Jakarta Barat,Super Indo,supermarket,-6.216578,106.765227,NaN,NaN
...,...,...,...,...,...,...,...,...,...
56161,56161,way/1159592008,Tulang Bawang,NaN,chemist,NaN,NaN,NaN,NaN
56162,56162,way/1160380538,Tulang Bawang,NaN,kiosk,NaN,NaN,NaN,NaN
56163,56163,way/1186182191,Tulang Bawang,NaN,yes,NaN,NaN,NaN,NaN
56164,56164,way/1186354872,Tulang Bawang,Alfamart,convenience,NaN,NaN,NaN,NaN


In [97]:
import pandas as pd
import numpy as np

# 1. Daftar mapping untuk mengubah nama kabupaten di OSM dataframe
revisi_nama_osm = {
    'Banjarbaru': 'Kota Banjar Baru',
    'Tanjungpinang': 'Kota Tanjung Pinang', 
    'Pangkalpinang': 'Kota Pangkal Pinang',
    'Kulonprogo': 'Kulon Progo',
    'Tulang Bawang': 'Tulangbawang',
    'Tolitoli': 'Toli-Toli'
}

# 2. Apply perubahan nama kabupaten di OSM dataframe
print("Sebelum perubahan - Data yang tidak match:")
for old_name in revisi_nama_osm.keys():
    count = (osm['kabupaten_kota'] == old_name).sum()
    print(f"{old_name}: {count} data")

# Ubah nama kabupaten yang tidak match
osm['kabupaten_kota'] = osm['kabupaten_kota'].replace(revisi_nama_osm)

print("\nSetelah perubahan:")
for new_name in revisi_nama_osm.values():
    count = (osm['kabupaten_kota'] == new_name).sum()
    print(f"{new_name}: {count} data")

# 3. Sekarang lakukan matching ulang untuk data yang sudah direvisi
# Standardisasi ulang nama kabupaten
osm['kabupaten_clean'] = osm['kabupaten_kota'].apply(standardize_kabupaten_name)

# Update Kode_Kabupaten untuk data yang sebelumnya tidak match
mask = osm['Kode_Kabupaten'].isna()
osm.loc[mask, 'Kode_Kabupaten'] = osm.loc[mask, 'kabupaten_clean'].map(kode_mapping)

# 4. Handle kasus khusus jika masih ada yang tidak match
manual_mapping_additional = {
    'KOTA BANJAR BARU': '63.72',
    'KOTA TANJUNG PINANG': '21.72', 
    'KOTA PANGKAL PINANG': '19.71',
    'KULON PROGO': '34.01',
    'TULANGBAWANG': '18.05',
    'TOLI-TOLI': '72.04'
}

for kabupaten_clean, kode_kab in manual_mapping_additional.items():
    mask = (osm['kabupaten_clean'] == kabupaten_clean) & (osm['Kode_Kabupaten'].isna())
    osm.loc[mask, 'Kode_Kabupaten'] = kode_kab

# 5. Hapus kolom temporary
osm = osm.drop(columns=['kabupaten_clean'])

# 6. Cek hasil akhir matching
print("\n=== HASIL AKHIR SETELAH REVISI ===")
print(f"Total data OSM: {len(osm)}")
print(f"Data dengan Kode_Kabupaten: {osm['Kode_Kabupaten'].notna().sum()}")
print(f"Data tanpa Kode_Kabupaten: {osm['Kode_Kabupaten'].isna().sum()}")
print(f"Success rate: {osm['Kode_Kabupaten'].notna().sum()/len(osm)*100:.2f}%")

# 7. Tampilkan data yang masih tidak match (jika ada)
remaining_unmatched = osm[osm['Kode_Kabupaten'].isna()]
if len(remaining_unmatched) > 0:
    print("\nData yang masih tidak match setelah revisi:")
    print(remaining_unmatched['kabupaten_kota'].value_counts())

# 8. Verifikasi untuk kabupaten yang direvisi
print("\n=== VERIFIKASI KABUPATEN YANG DIREVISI ===")
for new_name in revisi_nama_osm.values():
    data = osm[osm['kabupaten_kota'] == new_name]
    if not data.empty:
        kode = data['Kode_Kabupaten'].iloc[0]
        count = len(data)
        print(f"✓ {new_name}: {count} data → Kode: {kode}")
    else:
        print(f"✗ {new_name}: Tidak ditemukan data")

# 9. Tampilkan struktur final
print("\n=== STRUKTUR FINAL OSM ===")
print(osm.info())

# 10. Sample data untuk kabupaten yang direvisi
print("\n=== SAMPLE DATA YANG DIREVISI ===")
sample_revised = osm[osm['kabupaten_kota'].isin(revisi_nama_osm.values())]
if not sample_revised.empty:
    print(sample_revised[['kabupaten_kota', 'Kode_Kabupaten', 'name', 'category']].head(10))
else:
    print("Tidak ada data sample untuk kabupaten yang direvisi")

Sebelum perubahan - Data yang tidak match:
Banjarbaru: 127 data
Tanjungpinang: 74 data
Pangkalpinang: 45 data
Kulonprogo: 41 data
Tulang Bawang: 21 data
Tolitoli: 3 data

Setelah perubahan:
Kota Banjar Baru: 127 data
Kota Tanjung Pinang: 74 data
Kota Pangkal Pinang: 45 data
Kulon Progo: 41 data
Tulangbawang: 21 data
Toli-Toli: 3 data

=== HASIL AKHIR SETELAH REVISI ===
Total data OSM: 56166
Data dengan Kode_Kabupaten: 56166
Data tanpa Kode_Kabupaten: 0
Success rate: 100.00%

=== VERIFIKASI KABUPATEN YANG DIREVISI ===
✓ Kota Banjar Baru: 127 data → Kode: 63.72
✓ Kota Tanjung Pinang: 74 data → Kode: 21.72
✓ Kota Pangkal Pinang: 45 data → Kode: 19.71
✓ Kulon Progo: 41 data → Kode: 34.01
✓ Tulangbawang: 21 data → Kode: 18.05
✓ Toli-Toli: 3 data → Kode: 72.04

=== STRUKTUR FINAL OSM ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56166 entries, 0 to 56165
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unn

In [98]:
osm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56166 entries, 0 to 56165
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      56166 non-null  int64  
 1   id              56166 non-null  object 
 2   kabupaten_kota  56166 non-null  object 
 3   Kode_Kabupaten  56166 non-null  object 
 4   name            44723 non-null  object 
 5   category        56166 non-null  object 
 6   latitude        29646 non-null  float64
 7   longitude       29646 non-null  float64
 8   addr:city       5614 non-null   object 
 9   source          1830 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 4.3+ MB


In [99]:
osm.to_csv('poi_osm/fix/osm_poi_kode.csv')

In [100]:
osm

,Unnamed: 0,id,kabupaten_kota,Kode_Kabupaten,name,category,latitude,longitude,addr:city,source
0,0,node/411668893,Jakarta Barat,31.73,Hero,supermarket,-6.165670,106.759241,NaN,NaN
1,1,node/412686079,Jakarta Barat,31.73,Super Indo,supermarket,-6.165414,106.782067,NaN,NaN
2,2,node/1354554837,Jakarta Barat,31.73,NaN,atm,-6.189133,106.796257,NaN,NaN
3,3,node/1354554844,Jakarta Barat,31.73,CFC,restaurant,-6.189235,106.796378,NaN,NaN
4,4,node/1541706845,Jakarta Barat,31.73,Super Indo,supermarket,-6.216578,106.765227,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
56161,56161,way/1159592008,Tulangbawang,18.05,NaN,chemist,NaN,NaN,NaN,NaN
56162,56162,way/1160380538,Tulangbawang,18.05,NaN,kiosk,NaN,NaN,NaN,NaN
56163,56163,way/1186182191,Tulangbawang,18.05,NaN,yes,NaN,NaN,NaN,NaN
56164,56164,way/1186354872,Tulangbawang,18.05,Alfamart,convenience,NaN,NaN,NaN,NaN
